In [ ]:
# Tile generation adapted from: https://fractaldle.medium.com/satellite-images-deep-learning-spacenet-building-segmentation-a5d145a81c33

In [ ]:
#%pip install geopandas
#%pip install rasterio
#%pip install utm

##%%capture
#!apt update
#!apt install -y software-properties-common
#!apt install -y gpg
#!add-apt-repository -y ppa:ubuntugis/ppa && apt-get update
#!add-apt-repository -y ppa:nextgis/ppa && apt-get update 
#!apt-get install -y gdal-bin
#!apt-get install -y libgdal-dev pgp
#!export CPLUS_INCLUDE_PATH=/usr/include/gdal
#!export C_INCLUDE_PATH=/usr/include/gdal
#!apt-get install -y --reinstall build-essential
#!pip3 install "setuptools<58"
#!pip3 install --global-option=build_ext --global-option="-I/usr/include/gdal" GDAL==`gdal-config --version`

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, Point, MultiPolygon
from shapely.prepared import prep
import timeit
import shutil
from tqdm import tqdm
import rasterio
from rasterio.plot import show_hist
from rasterio.plot import show
from rasterio.windows import Window
import utm
import pyproj
import os
import pandas as pd
import geopandas as gpd
import random
from osgeo import gdal
import time

from sentinel2_transform import UTM2LonLat, get_lon_lat_bounds, get_filename, get_tile_transform, get_tile_profile, get_countries, get_utm_zone_polygon, all_in_utm_zone, generate_tiles      

In [ ]:
# Set session and s3 connection objects
sess = sagemaker.Session()
role = get_execution_role()
conn = boto3.client("s3")
bucket = "w210-poverty-mapper"
s3_path = "sentinel2_composite/transformed_data"

In [ ]:
country_shapes = gpd.read_file("s3://w210-poverty-mapper/shape_files/raw_data/ipums_world_shapes/world_countries_2020.shp")

In [ ]:
country_list = ["Bangladesh", "Nepal", "Philippines", "Tajikistan", "Timor Leste"]
polygon_dict = {}

for country in country_list:
    polygon = country_shapes.loc[country_shapes["CNTRY_NAME"] == country]["geometry"].iloc[0]
    polygon_dict[country] = prep(polygon) # Prepare the polygon for faster operations later on

In [ ]:
zone_list = ["42S", "42T", 
             "43S", 
             "44R", 
             "45Q", "45R", 
             "46Q", "46R", 
             "50N", "50P", "50Q", 
             "51L", "51N", "51P", "51Q", 
             "52L", "52N", "52P"]
size = 224

generate_tiles(conn, bucket, zone_list, size, polygon_dict, s3_path)